In [1]:
import numpy as np
import time
import sys

class Conv3x3:
    
    def __init__(self, num_filters,f,n_c):
        self.num_filters = num_filters
        self.n_c = n_c
        self.f = f
        self.filters = np.random.normal(loc=0, scale=np.sqrt(1./(self.n_c*self.f*self.f)), size=(num_filters,self.n_c, self.f, self.f))
        self.t_f = 0
    
    def forward(self, input):
        
        u = time.time()
        
        self.last_input = input
    
        h, w , n_c = input.shape
        
        
        output = np.zeros((h - self.f+1, w - self.f+1, self.num_filters),dtype=np.float64)
    
        for i in range(h-self.f+1):
            for j in range(w-self.f+1):
                for k in range(self.num_filters):
                    for n in range(self.n_c):
                        #print("n",n)
                        imr = input[i:i+self.f,j:j+self.f,n]
                        for p in range(self.f):
                            for q in range(self.f):
                                output[i, j , k] += np.sum( imr[p,q] * self.filters[k,n,p,q])
                            
                    output[i,j,k] = np.maximum(0,output[i,j,k])
                    
        self.last_output = output

        v =  time.time()
        
        self.t_f += v-u
        
        return output
    
    def backprop(self, d_L_d_out, learn_rate):
        
        d_L_d_out[self.last_output<0] = 0
        
        d_L_d_filters = np.zeros(self.filters.shape,dtype=np.float64)
    
        h,w,c = self.last_input.shape
        d_L_d_i = np.zeros((h,w,c))
        for i in range(h-self.f+1):
            for j in range(w-self.f+1):
                  for f in range(self.num_filters):
                        for n in range(self.n_c):
                            d_L_d_filters[f,n,:,:] += d_L_d_out[i, j, f] * self.last_input[i:i+self.f,j:j+self.f,n]
                            d_L_d_i[i:i+self.f,j:j+self.f,n] += d_L_d_out[i, j, f] * self.filters[f,n,:,:]
                            
        self.filters -= learn_rate * d_L_d_filters

        return d_L_d_i


In [2]:
import numpy as np


class MaxPool2:
    
    def iterate_regions(self, image):
        h, w, _ = image.shape
        new_h = h // 2
        new_w = w // 2
    
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i * 2):(i * 2 + 2), (j * 2):(j * 2 + 2)]
                yield im_region, i, j
    
    def forward(self, input):
    
        self.last_input = input

        h, w, num_filters = input.shape
        output = np.zeros((h // 2, w // 2, num_filters))

        for im_region, i, j in self.iterate_regions(input):
            output[i, j] = np.amax(im_region, axis=(0, 1))

        return output

    def backprop(self, d_L_d_out):
       
        d_L_d_input = np.zeros(self.last_input.shape)

        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            amax = np.amax(im_region, axis=(0, 1))

            for i2 in range(h):
                for j2 in range(w):
                      for f2 in range(f):
                        # If this pixel was the max value, copy the gradient to it.
                            if im_region[i2, j2, f2] == amax[f2]:
                                  d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]

        return d_L_d_input

In [3]:
import numpy as np
from copy import deepcopy

class fc_relu:

    def __init__(self, input_len, nodes):
        self.weights = np.random.randn(input_len, nodes)*np.sqrt(2/nodes,dtype=np.float64)
        self.weights[self.weights<-1] = -1
        self.weights[self.weights>1] = 1
        self.biases = np.zeros(nodes,dtype=np.float64)

    def forward(self, input):
        
        self.last_input_shape = input.shape

        input = input.flatten()
        self.last_input = input

        input_len, nodes = self.weights.shape

        totals = np.dot(input, self.weights) + self.biases
        self.last_totals = totals

        
        return totals
    
    
        
        
    def backprop(self, d_L_d_out, learn_rate):
        
        d_L_d_t = deepcopy(d_L_d_out)
        
        d_L_d_t[self.last_totals<0] = 0
        
        d_t_d_w = self.last_input
        d_t_d_b = 1
        d_t_d_inputs = self.weights
        
        d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
        d_L_d_b = d_L_d_t * d_t_d_b
        d_L_d_inputs = d_t_d_inputs @ d_L_d_t

        self.weights -= learn_rate * d_L_d_w
        self.biases -= learn_rate * d_L_d_b

        return d_L_d_inputs.reshape(self.last_input_shape)


In [4]:
import numpy as np

class Softmax:

    def __init__(self, input_len, nodes):
        self.weights = np.random.normal(loc=0, scale=np.sqrt(1./(input_len*nodes)), size=(input_len,nodes))
        self.biases = np.random.normal(loc=0, scale=np.sqrt(1./(nodes)), size=(nodes))

    def forward(self, input):
        
        
        self.last_input_shape = input.shape

        input = input.flatten()
        
        self.last_input = input

        input_len, nodes = self.weights.shape[0],self.weights.shape[1]

        totals = np.dot(input, self.weights) + self.biases
        
        totals -= np.max(totals)
        
        self.last_totals = totals
        
        exp = np.exp(totals,dtype=np.float64)
        
        return exp / np.sum(exp, axis=0)

    def backprop(self, d_L_d_out, learn_rate):
        
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue

            t_exp = np.exp(self.last_totals)

            S = np.sum(t_exp)

            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)
            
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights

            d_L_d_t = gradient * d_out_d_t

            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t

            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b

            return d_L_d_inputs.reshape(self.last_input_shape) 

In [5]:
from mnist import MNIST

mndata = MNIST(r'C:\Users\krish\OneDrive\Documents\krishna\sem6\DL\mnist')

mndata.gz = True

train_images = np.zeros([400,28,28,1])
test_images = np.zeros([100,28,28,1])


images_tr,train_labels = mndata.load_training()
images_tr = np.array(images_tr[:400])
train_labels = np.array(train_labels[:400])
    

images_ts1,test_labels = mndata.load_testing()
images_ts = np.array(images_ts1[:100])
test_labels = np.array(test_labels[0:100])


for i in range(400):
    train_images[i] = images_tr[i].reshape(28,28,1) 
    
for i in range(100):    
    test_images[i] = images_ts[i].reshape(28,28,1)
    
#print(train_images[0])

In [6]:
import mnist
import numpy as np
import tqdm as tqdm

conv1 = Conv3x3(8,3,1)
pool1 = MaxPool2()

conv2 = Conv3x3(8,3,1)
pool2 = MaxPool2()

#fc_r = fc_relu(200,84)

softmax = Softmax(200, 10) 

def forward(image, label): 
    
    out = conv1.forward((image/255) )
    out = pool1.forward(out)
    out = conv2.forward(out)
    out = pool2.forward(out)
    #out = fc_r.forward(out)
    out = softmax.forward(out)
    
    if out[label] == 0:
        out[label] = sys.float_info.min
    
    
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0

    return out, loss, acc

def train(im, label, lr=.005):
   
    out, loss, acc = forward(im, label)

    gradient = np.zeros(10)
    if out[label] == 0:
        out[label] = sys.float_info.min
    gradient[label] = -1 / out[label]

    gradient = softmax.backprop(gradient, lr)
    #gradient = fc_r.backprop(gradient, lr)
    gradient = pool2.backprop(gradient)
    gradient = conv2.backprop(gradient, lr)
    gradient = pool1.backprop(gradient)
    gradient = conv1.backprop(gradient, lr)
    
    return loss, acc

t1 = time.time()
for epoch in range(3):
#     break
    print('\n')
    print('--- Epoch %d ---' % (epoch + 1))

    permutation = np.random.permutation(len(train_images))
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]
    
    loss = 0
    num_correct = 0
    outer = tqdm.tqdm(total=len(train_images), desc='training_data', position=0)
    
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        outer.update(1)
        l, acc = train(im, label)
        loss += l
        num_correct += acc
        
        
        
    print('after epoch' + str(epoch+1) + ' loss is ' + str(loss/len(train_images)) + ' accuracy is ' + str(num_correct/len(train_images)))
    
    
t2 = time.time()

total_time = t2-t1

total_time_conv = conv1.t_f 
        
        
print('Testing on test data set')
loss = 0
num_correct = 0

test = tqdm.tqdm(total=len(test_images), desc='test_data', position=0)
    
for im, label in zip(test_images, test_labels):
    test.update(1)
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc



num_tests = len(test_images)
print('Loss on test dataset:', loss / num_tests)
print('Accuracy on test dataset:', num_correct / num_tests)
print('\n')
print('total time taken is ',total_time)
print('total time taken for conv forward prop is ',total_time_conv)




--- Epoch 1 ---


training_data: 100%|█████████████████████████████████████████████████████████████████| 400/400 [04:02<00:00,  1.55it/s]

after epoch1 loss is 2.242060804192134 accuracy is 0.1675


--- Epoch 2 ---


training_data: 100%|█████████████████████████████████████████████████████████████████| 400/400 [04:09<00:00,  1.55it/s]

after epoch2 loss is 1.1644775295468663 accuracy is 0.6525


--- Epoch 3 ---


training_data: 100%|█████████████████████████████████████████████████████████████████| 400/400 [03:53<00:00,  1.53it/s]

after epoch3 loss is 0.6700316495859814 accuracy is 0.785
Testing on test data set


test_data: 100%|█████████████████████████████████████████████████████████████████████| 100/100 [00:53<00:00,  1.79it/s]

Loss on test dataset: 0.7951450118170683
Accuracy on test dataset: 0.77


total time taken is  726.9290916919708
total time taken for conv forward prop is  503.93260526657104


In [7]:
from random import randint
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow, show

def predict_out(image):

    out = conv1.forward((image / 255) - 0.5)
    out = pool1.forward(out)
    out = Softmax(1352,10).forward(out)
    
    return np.argmax(out)



print("test cases: \n\n ")

for j in range(7):
    rand = randint(1000,2000)
    print("\n\ninput image :")
    test_img = np.array(images_ts1[rand]).reshape(28,28)
    imshow(test_img)
    show()
    
    print(" predicted Output" , end = "")
    print(predict_out(test_img.reshape(28,28,1)))
    

test cases: 

 


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output2


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output5


input image :


...

 predicted Output4


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output5


input image :


<Figure size 640x480 with 1 Axes>

 predicted Output0


In [8]:
Loss on test dataset: 0.9290618471892992
Accuracy on test dataset: 0.74


total time taken is  1669.1526427268982
total time taken for conv forward prop is  599.6809167861938

SyntaxError: invalid syntax (<ipython-input-8-7b0e9b52719b>, line 1)

In [ ]:
Accuracy on test dataset: 0.77


total time taken is  726.9290916919708
total time taken for conv forward prop is  503.93260526657104